In [1]:
import collections
import copy
import datetime
import itertools

import json
import pathlib
import re
from pprint import pprint

import nest_asyncio
nest_asyncio.apply()
import texttable

import pytourney
import pyfumbbl
import cibblbibbl

In [2]:
G = cibblbibbl.CIBBL
Achievement = cibblbibbl.achievement.Achievement
Team = cibblbibbl.team.Team
player = cibblbibbl.player.player
Match = cibblbibbl.match.Match
Matchup = cibblbibbl.matchup.Matchup
Replay = cibblbibbl.replay.Replay

In [5]:
T = G.tournaments["44753"]

In [4]:
print(cibblbibbl.tournament.export.awards.plaintext.export(T))

8th place: Trantio Dragon Hawks
Prestige Points Earned: 10 (and -80 Achiev.)

7th place: Nejaz Djinn
Prestige Points Earned: 70 (and -10 Achiev.)

6th place: Secret Cult Of Shiamoto
Prestige Points Earned: 100 (and +20 Achiev.)

5th place: Everlasting Vultures of Settra
Prestige Points Earned: 100 (and -30 Achiev.)

4th place: Tor Anroc Anglers
Prestige Points Earned: 130 (and +50 Achiev.)

3rd place: Tower of the Necromancer
Prestige Points Earned: 180

2nd place: Talsyn Tramplers
Prestige Points Earned: 200 (and +50 Achiev.)

1st place: Montetriste Noblemen
Prestige Points Earned: 250 (and +40 Achiev.)


=== Gold Excellence (10) ===
Acurag (Talsyn Tramplers) (Fouls)
Count Yaroslav (Tower of the Necromancer) (Blocking Scorer) (Achievement already earned)
Irtriss (Everlasting Vultures of Settra) (SPP)
Manuel Grandegolpe (Montetriste Noblemen) (Casualties)
Nuirlan (Tor Anroc Anglers) (Completions)
Telemaco Di Domenico (Trantio Dragon Hawks) (Interceptions)
Velasco Estibador (Montetriste

In [46]:
T.standings()

[{'cas': 15,
  'casdiff': 6,
  'score': 10,
  'scorediff': 8,
  'td': 10,
  'tddiff': 8,
  'pts': 9,
  'prestige': 130,
  'cto': -1,
  'hth': -1,
  'perf': [('D', 4010177),
   ('W', 4013422),
   ('W', 4014253),
   ('W', 4018986),
   ('W', 4019586)],
  'team': Team(912296)},
 {'cas': 8,
  'casdiff': 4,
  'score': 6,
  'scorediff': 3,
  'td': 6,
  'tddiff': 3,
  'pts': 7,
  'prestige': 100,
  'cto': -1,
  'hth': -1,
  'perf': [('D', 4011497),
   ('W', 4013157),
   ('W', 4015218),
   ('L', 4018986),
   ('W', 4019995)],
  'team': Team(914971)},
 {'cas': 3,
  'casdiff': -7,
  'score': 8,
  'scorediff': 5,
  'td': 4,
  'tddiff': 1,
  'pts': 6,
  'prestige': 90,
  'cto': -1,
  'hth': 0,
  'perf': [('L', 4010424),
   ('B', None),
   ('B', None),
   ('L', 4018374),
   ('W', 4019150)],
  'team': Team(913702)},
 {'cas': 13,
  'casdiff': 7,
  'score': 6,
  'scorediff': -1,
  'td': 6,
  'tddiff': -1,
  'pts': 6,
  'prestige': 90,
  'cto': -1,
  'hth': 0,
  'perf': [('W', 4012186),
   ('L', 4013157)

In [47]:
dPAv1 = T.playerachievementvalues()
dPAv1

{Team(912296): 0,
 Team(914971): 0,
 Team(913702): 25,
 Team(914538): 25,
 Team(912837): 10,
 Team(914744): 15,
 Team(912821): 0,
 Team(914618): 10,
 Team(913011): 10,
 Team(912165): 0}

In [48]:
dPP = T.playerperformances()

In [49]:
achievements = sorted(T.achievements)
d_achievements = collections.defaultdict(dict)
for a in achievements:
    d_achievements[a.clskey()][a.subject] = a

In [50]:
prev_tournament = {}
for Te in T.teams:
    prev_tournament[Te] = Te.prev_tournament(T)

In [36]:
nrsuffix = {1: "st", 2: "nd", 3: "rd"}
standings = list(enumerate(T.standings(), 1))
for nr, d in reversed(standings):
    Te = d["team"]   
    nrstr = f'{nr}{nrsuffix.get(nr, "th")}'
    print(f'{nrstr} place: {Te.name}')
    TP_matchv, TP_standingsv = 0, 0
    TP_match = d_achievements.get("tp_match", {}).get(Te)
    if TP_match:
      TP_matchv = TP_match.prestige(T.season)
    TP_standings = d_achievements.get("tp_standings").get(Te)
    if TP_standings:
      TP_standingsv = TP_standings.prestige(T.season)
    preststr = f'Prestige Points Earned: {TP_matchv+TP_standingsv}'
    dTPAv1 = dPAv1[Te]
    T0 = prev_tournament[Te]
    if T0:
        dPAv0 = T0.playerachievementvalues()
        dTPAv0 = dPAv0[Te]
    else:
        dTPAv0 = 0
    if dTPAv1 - dTPAv0:
        preststr += f' (and {("+" if -1 < dTPAv1 - dTPAv0 else "")}{dTPAv1 - dTPAv0} Achiev.)'
    print(preststr)
    print()

12th place: Cothique Wanderers
Prestige Points Earned: 20 (and +10 Achiev.)

11th place: Black Hearted Bloodseekers
Prestige Points Earned: 30 (and +10 Achiev.)

10th place: Seekers of Transcendence
Prestige Points Earned: 40

9th place: Ragtag Animal Training Sartors
Prestige Points Earned: 50 (and +10 Achiev.)

8th place: K'r'shh Sl'ppr'ss
Prestige Points Earned: 70

7th place: Black Monolith Worshippers
Prestige Points Earned: 70 (and -20 Achiev.)

6th place: Norscan Curse
Prestige Points Earned: 70

5th place: Sport Spawning Zlatlan
Prestige Points Earned: 70

4th place: BloodForce
Prestige Points Earned: 110 (and +10 Achiev.)

3rd place: Strigos Elită
Prestige Points Earned: 130 (and +30 Achiev.)

2nd place: Deluge-Bearers
Prestige Points Earned: 160 (and +30 Achiev.)

1st place: Horsemen of The Great Kurgan
Prestige Points Earned: 190



In [51]:
prev_clskey = None
for i, A in enumerate(sorted(A for A in T.achievements if not A.clskey().startswith("tp"))):
    clskey = A.clskey()
    if clskey != prev_clskey:
        if i:
            print()
        print(f'=== {A["name"]} ({A.baseprestige}) ===')
    if clskey.startswith("ta"):
        s = f'{A.subject.name}'
    else:
        s = f'{A.subject.name} ({dPP[A.subject]["team"].name})'
        if clskey in ("pa_bewaresupremekiller", "pa_targeteliminated"):
            victim = cibblbibbl.player.player(A["victimId"])
            s += f' ({A["reason"]} {victim.name})'
    try:
        categs = A["categs"]
    except KeyError:
        pass
    else:
        s += f' ({", ".join(sorted(categs))})'
    stack = A.stack(T.season)
    if 1 < len(stack):
        statidx = A.stackidx()
        p0 = sum(A1.prestige(T.season.prev) for A1 in stack if A1 is not A)
        p1 = sum(A1.prestige(T.season) for A1 in stack)
        dp = p1 - p0
        if dp:
            dpstr = f' {("+" if -1 < dp else "")}{dp}'
        else:
            dpstr = ""
        s += f' (Achievement already earned{dpstr})'
    dpp = dPP[a.subject]
    if dpp.get("dead"):
        s += " (posthumous)"
    elif dpp.get("retired") and not dpp.get("retiredlast"):
        s += " (retired)"
    #print(statidx, A.prestige(T.season), s)        
    print(s)
    prev_clskey = clskey 

=== SCRIBBL’s Outstanding Performance Achievement (15) ===
Evrart le Prestre (Bordeleaux Chevaliers) (Completions)
Girim Shattermantle (Mugrec's Mechanical Madmen) (Fouls)
O'rdra The Deathmaker (BloodForce) (SPP)

=== Mean Scoring Machine (10) ===
Phelippon Pidoye (Bordeleaux Chevaliers)

=== Triple X (10) ===
Gnome Defence Alternator Apparatus (Gnoming (2509))
Krooksnaggah ('Ed Chompaz of Deff Gorge)
O'rdra The Deathmaker (BloodForce)
Ruk Stonebender (Kundun Soulbutchers)


In [45]:
T = G.tournaments["45586"]

In [7]:
Ma = Match(3981578)

In [8]:
Mu = Ma.matchup

In [9]:
Mu.tournament

Tournament('cibbl', '44753')

In [10]:
Mu.tournament.achievements

{TP_Match('tp_match', Tournament('cibbl', '44753'), Team(854600)),
 TP_Match('tp_match', Tournament('cibbl', '44753'), Team(869156)),
 TP_Match('tp_match', Tournament('cibbl', '44753'), Team(871312)),
 TP_Match('tp_match', Tournament('cibbl', '44753'), Team(867862)),
 TP_Match('tp_match', Tournament('cibbl', '44753'), Team(854690)),
 TP_Match('tp_match', Tournament('cibbl', '44753'), Team(863441)),
 TP_Match('tp_match', Tournament('cibbl', '44753'), Team(855753)),
 TP_Match('tp_match', Tournament('cibbl', '44753'), Team(855120)),
 TP_Standings('tp_standings', Tournament('cibbl', '44753'), Team(854600)),
 TP_Standings('tp_standings', Tournament('cibbl', '44753'), Team(869156)),
 TP_Standings('tp_standings', Tournament('cibbl', '44753'), Team(871312)),
 TP_Standings('tp_standings', Tournament('cibbl', '44753'), Team(863441)),
 PA_BPA_Gold('pa_bpa_gold', Tournament('cibbl', '44753'), NormalPlayer('11667471')),
 PA_BPA_Gold('pa_bpa_gold', Tournament('cibbl', '44753'), NormalPlayer('1181290

In [12]:
Pl = player("11750105")

In [13]:
Pl.achievements

{PA_BPR_OldWorld('pa_bpr_oldworld', Tournament('cibbl', '44070'), NormalPlayer('11750105'))}

In [11]:
T.deadplayers()

{NormalPlayer('11781773'): [3987503, 1, 2, 'blocked', '11582132'],
 NormalPlayer('11700319'): [3989129, 1, 2, 'blocked', '12264876'],
 NormalPlayer('11700323'): [3991062, 2, 4, 'blocked', '11582136'],
 NormalPlayer('11850036'): [3977459, 1, 7, 'blocked', '11667458'],
 NormalPlayer('11775641'): [3982439, 1, 7, 'blocked', '12152809'],
 NormalPlayer('11979086'): [3987334, 2, 2, 'blocked', '11439814'],
 NormalPlayer('12135210'): [3977527, 1, 2, 'blocked', '12004052'],
 NormalPlayer('11446518'): [3977772, 1, 3, 'blocked', '11455997'],
 NormalPlayer('11787823'): [3984706, 1, 5, 'blocked', '11667459'],
 NormalPlayer('11921960'): [3984706, 2, 3, 'blocked', '11667459'],
 NormalPlayer('12178154'): [3989079, 2, 5, 'blocked', '11439814'],
 NormalPlayer('11456001'): [3986830, 2, 6, 'blocked', '11667462'],
 NormalPlayer('11502188'): [3978318, 1, 3, 'blocked', '11646894'],
 NormalPlayer('12197779'): [3991585, 2, 5, 'blocked', '11667458'],
 NormalPlayer('11646902'): [3985702, 1, 8, 'blocked', '1158213

In [52]:
dPP

{NormalPlayer('12365073'): {'blocks': 21,
  'cas': 1,
  'fouls': 0,
  'td': 0,
  'mvp': 0,
  'rush': 0,
  'int': 0,
  'pass': 0,
  'spp': 2,
  'comp': 0,
  'turns': 29,
  'perf': [('L', 4009868), ('L', 4013422)],
  'team': Team(912837),
  'name': 'The Gnomissh’ Thingamajig',
  'type': 'B'},
 NormalPlayer('12365074'): {'blocks': 22,
  'cas': 3,
  'fouls': 0,
  'td': 0,
  'mvp': 0,
  'rush': 0,
  'int': 0,
  'pass': 0,
  'spp': 6,
  'comp': 0,
  'turns': 31,
  'perf': [('L', 4009868), ('L', 4013422)],
  'team': Team(912837),
  'name': 'Gnome Defence Alternator Apparatus',
  'type': 'B'},
 NormalPlayer('12365075'): {'blocks': 4,
  'cas': 0,
  'fouls': 0,
  'td': 0,
  'mvp': 0,
  'rush': 0,
  'int': 0,
  'pass': 0,
  'spp': 0,
  'comp': 0,
  'turns': 10,
  'perf': [('L', 4009868), ('L', 4013422)],
  'team': Team(912837),
  'name': 'Totxixc Jr.',
  'type': 'R'},
 NormalPlayer('12365076'): {'blocks': 12,
  'cas': 0,
  'fouls': 0,
  'td': 0,
  'mvp': 1,
  'rush': 3,
  'int': 0,
  'pass': 1,
 